# Simple Patch Level Convolutional Neural Network 

### This is just a framework and is not yet functional

In [6]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

%matplotlib inline

In [5]:
batchSize = 20
patch_size = 5
depth = 16
num_hidden = 64
numb_labels = 10
image_size = 256
num_channels = 3

graph = tf.Graph()

with graph.as_default():
    
    conv_tf_train_data = tf.placeholder(
        tf.float32, shape = (batchSize, image_size, image_size, num_channels))
    conv_tf_train_labels = tf.placeholder(
        tf.float32, shape = (batchSize, numb_labels))
    conv_tf_valid_data = tf.Variable(conv_valid_data)
    conv_tf_test_data = tf.Variable(conv_test_data)
    
    
    weights_conv_01 = tf.Variable(
        tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev= 0.1))
    biases_conv_01 = tf.Variable(
        tf.zeros([depth]))
    weights_conv_12 = tf.Variable(
        tf.truncated_normal([patch_size, patch_size, depth, depth]))
    biases_conv_12 = tf.Variable(
        tf.constant(1.0, shape = [depth]))
    weights_connected_23 = tf.Variable(
        tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    biases_connected_23 = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    weights_connected_34 = tf.Variable(
        tf.truncated_normal([num_hidden, numb_labels], stddev=0.1))
    biases_connected_34 = tf.Variable(tf.constant(1.0, shape=[numb_labels]))

    
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, weights_conv_01, [1, 2, 2, 1], padding='SAME')  # Stride distance of 2
        hidden = tf.nn.relu(conv + biases_conv_01)
        conv = tf.nn.conv2d(hidden, weights_conv_12, [1, 2, 2, 1], padding='SAME') # Stride distance of 2
        hidden = tf.nn.relu(conv + biases_conv_12)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, weights_connected_23) + biases_connected_23)
        return tf.matmul(hidden, weights_connected_34) + biases_connected_34

    # Training computation.
    logits = model(conv_tf_train_data)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, conv_tf_train_labels))

    # Optimizer.
    optimiser = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(conv_tf_valid_data))
    test_prediction = tf.nn.softmax(model(conv_tf_test_data))

NameError: name 'conv_valid_data' is not defined

In [ ]:
num_steps = 3301
graph_minibatch = []
graph_valid = []
steps = []

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
    
    tf.initialize_all_variables().run()
    
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batchSize) % (train_labels.shape[0] - batchSize)
        batch_data = conv_train_data[offset:(offset + batchSize), :, :, :]
        batch_labels = train_labels[offset:(offset + batchSize), :]
        feed_dict = {conv_tf_train_data : batch_data, conv_tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimiser, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            steps.append(step)
            graph_minibatch.append(accuracy(predictions, batch_labels))
            graph_valid.append(accuracy(valid_prediction.eval(), valid_labels))
        if (step % 600 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))